Step ! importing Modules

In [55]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits

Step @ creating Dataframe

In [61]:
df=load_digits()
X=df.data
Y=df.target
x=pd.DataFrame(X)
y=pd.DataFrame(Y)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=17)
x_train.shape, y_train.shape



((1437, 64), (1437, 1))

Step # create neural net class

In [62]:
class NeuralNet():
    model=keras.Sequential([
        keras.layers.Dense(128, input_shape=(64,), activation='relu'),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    def fit(self, x_train, y_train, epochs=10):
        self.model.fit(x_train, y_train, epochs=epochs)
    def evaluate(self, x_test, y_test):
        return self.model.evaluate(x_test, y_test)
    def predict(self, x):
        return self.model.predict(x)

e:\Projects\VSC\.venv\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [63]:
NN=NeuralNet()
NN.fit(x_train, y_train, epochs=50)
NN.evaluate(x_test, y_test)

Epoch 1/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4499 - loss: 1.9388   
Epoch 2/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9167 - loss: 0.3713 
Epoch 3/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9622 - loss: 0.1620 
Epoch 4/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9833 - loss: 0.0887 
Epoch 5/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9868 - loss: 0.0653 
Epoch 6/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9964 - loss: 0.0338 
Epoch 7/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9969 - loss: 0.0314 
Epoch 8/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9990 - loss: 0.0219
Epoch 9/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9980 - loss: 0.0187 
Epoch 10/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9972 - loss: 0.0140 
Epoch 11/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9949 - loss: 0.0203 
Epoch 12/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy:

[0.08064290881156921, 0.9722222089767456]

Step $ create a custom NN of 1 hidden layer

In [108]:
def sigmoid(x):
        return (1/(1+np.exp(-x)))
def logloss(y_true,y_pred):
    epsilon=1e-15
    y_pred_new=[max(i,epsilon) for i in y_pred]
    y_pred_new=[min(i,1-epsilon) for i in y_pred_new]
    y_pred_new=np.array(y_pred_new)
    ll=-np.mean(y_true*np.log(y_pred_new)+(1-y_true)*np.log(1-y_pred_new))
    return ll
class CustomNN():
    # Corrected __init__ method
    def __init__(self, x, y,layers,layers_dim, learning_rate=0.01):
        self.layers=layers
        self.layers_dim=layers_dim
        self.lr = learning_rate
        self.n_samples, self.n_features = x.shape  # Correctly unpack shape
        self.weights = [np.random.randn(self.layers_dim[i], self.layers_dim[i+1]) for i in range(self.layers - 1)]
        print("Weight matrix shapes:")
        for w in self.weights:
            print(w.shape)
        self.bias=[np.zeros((1,self.layers_dim[i+1])) for i in range(self.layers-1)]
        print("Bias vector shapes:")
        for b in self.bias:
            print(b.shape)
        self.x = x
        self.y = y.reshape(-1, 1) # Reshape y to be a column vector
        #print(self.x.shape, self.wt.shape)
    def forward(self):
        # Store initial input as the first activation
        self.activations = [self.x]
        
        # Loop through each layer, starting from the first hidden layer
        for i in range(self.layers - 1):
            # The input to this layer is the activation from the previous layer
            input_layer = self.activations[i]
            
            # Calculate the linear combination (z) and apply activation (a)
            z = np.dot(input_layer, self.weights[i]) + self.bias[i]
            a = sigmoid(z)
            
            # Store the new activation for the next layer's input
            self.activations.append(a)
        
        # The final activation is the network's prediction
        y_pred = self.activations[-1]
    
        return y_pred
    def backward(self, y_pred):
        # Backpropagation starts with the error at the output layer
        error = y_pred - self.y
        
        # Calculate gradients for the last layer (output layer)
        # dw for the last layer: (Input_last_hidden_layer.T @ error)
        dw = (1 / self.n_samples) * np.dot(self.activations[-2].T, error)
        db = (1 / self.n_samples) * np.sum(error, axis=0, keepdims=True)

        # Update weights and bias for the last layer
        self.weights[-1] = self.weights[-1] - self.lr * dw
        self.bias[-1] = self.bias[-1] - self.lr * db
        
        # Now, loop backward through the hidden layers
        for i in range(len(self.weights) - 2, -1, -1):
            # Propagate the error backward using the chain rule
            error = np.dot(error, self.weights[i+1].T) * (self.activations[i+1] * (1 - self.activations[i+1]))
            
            # Calculate gradients for the current hidden layer
            dw = (1 / self.n_samples) * np.dot(self.activations[i].T, error)
            db = (1 / self.n_samples) * np.sum(error, axis=0, keepdims=True)
            
            # Update weights and bias
            self.weights[i] = self.weights[i] - self.lr * dw
            self.bias[i] = self.bias[i] - self.lr * db


    def fit(self, epochs=100):
        for i in range(epochs):
            y_pred=self.forward()
            self.backward(y_pred)
            if i%10==0:
                print(f"epoch {i} logloss: {logloss(self.y,y_pred)}")
    def predict(self, x_test):
    # Start with the test data as the initial activation
        activations = [x_test]
    
        # Perform a full forward pass through all layers
        for i in range(len(self.weights)):
            z = np.dot(activations[i], self.weights[i]) + self.bias[i]
            a = sigmoid(z)
            activations.append(a)
        
        y_pred = activations[-1]
        y_pred_cls = np.array([1 if i > 0.5 else 0 for i in y_pred.flatten()])
        return y_pred_cls

In [109]:
#print(y_train.shape)
layers_dim=[64,32,16,8,1]
cNN=CustomNN(x_train.values,y_train.values,5,layers_dim, learning_rate=0.01)
cNN.fit(epochs=100)
y_pred=cNN.predict(x_test.values)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)


Weight matrix shapes:
(64, 32)
(32, 16)
(16, 8)
(8, 1)
Bias vector shapes:
(1, 32)
(1, 16)
(1, 8)
(1, 1)
epoch 0 logloss: 17.432495483946127
epoch 10 logloss: 3.614694260718764
epoch 20 logloss: -5.302394654573595
epoch 30 logloss: -10.68394155105929
epoch 40 logloss: -15.156722098088604
epoch 50 logloss: -19.506474417207222
epoch 60 logloss: -24.008291873203337
epoch 70 logloss: -28.704781014747173
epoch 80 logloss: -33.531260487889774
epoch 90 logloss: -38.42045274560896


0.09444444444444444